## Create Database from df

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import pandas as pd
import json

In [3]:
# configure
base_dir = "data/"
# Path to your SQLite database file
database_file_path = f"{base_dir}/db/recipes.db"

## Create Dabatase

## Prepare the SQL prompt

In [4]:
import os
import openai
# from IPython.display import Markdown, HTML, display
# from llama_index.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
# from langchain_openai import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI


In [5]:
RECIPE_AGENT_PREFIX = """

You are an agent designed to interact with a database of recipes.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it. If you get an error
while executing a query, rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [6]:
RECIPE_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

## Important Notes:
- Always include all columns in your SQL query except `general_info.link`.
- Present the final answer in a structured format, following the template provided below.

## Example of Final Answer:

Here are 3 recipes that include chicken:

1. **Creamy Tuscan Chicken**:
   - **Prep time**: 10 mins
   - **Cook time**: 25 mins
   - **Difficulty**: Easy
   - **Serves**: 4
   - **Description**: This creamy Tuscan chicken is delicious and easy to make.
   - **Diet Type**: Not specified
   - **Nutrition (per serving)**:
     - **Calories**: 672 kcal
     - **Fat**: 55g
     - **Saturates**: 23g
     - **Carbs**: 12g
     - **Sugars**: 5g
     - **Fibre**: 2g
     - **Protein**: 35g
     - **Salt**: 1.5g
   - **Ingredients**:
     - 4 chicken breasts
     - 2 tablespoons olive oil
     - 1/2 cup heavy cream
     - 1/4 cup sun-dried tomatoes
     - 2 cloves garlic
     - 1/2 cup chicken broth
   - **Instructions**:
     1. Heat the olive oil in a large skillet over medium heat.
     2. Add the chicken breasts and cook until golden brown.
     3. Add the garlic and sun-dried tomatoes, cook for another minute.
     4. Pour in the chicken broth and cream, then simmer until the sauce thickens.
     5. Serve hot with your choice of sides.

Explanation:
I queried the `recipes` table for all relevant columns except `general_info.link` where the ingredients include 'chicken'. The query returned a list of recipes with detailed information including prep and cook times, nutrition details, ingredients, and instructions. I used the following query:

```sql
SELECT [general_info.title], [general_info.prep_time], [general_info.cook_time], [general_info.difficulty], [general_info.serves], [general_info.description], [general_info.diet_type], [nutrition_info.kcal], [nutrition_info.fat], [nutrition_info.saturates], [nutrition_info.carbs], [nutrition_info.sugars], [nutrition_info.fibre], [nutrition_info.protein], [nutrition_info.salt], [ingredients], [instructions]
FROM recipes 
WHERE [ingredients] LIKE '%chicken%'
LIMIT 3
"""

## Call the Chat model and create the SQL agent

In [7]:

# Obtén la clave API de OpenAI desde las variables de entorno
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=openai.api_key)

db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
QUESTION = "givme a recipe with chicken in and spaghetti in it"

agent_executor_SQL = create_sql_agent(
    prefix=RECIPE_AGENT_PREFIX,
    format_instructions = RECIPE_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [13]:
response = agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: recipesI should query the schema of the recipes table to see what columns are available for me to use in my query.
Action: sql_db_schema
Action Input: recipes
CREATE TABLE recipes (
	"general_info.link" TEXT, 
	"general_info.title" TEXT, 
	"general_info.prep_time" TEXT, 
	"general_info.cook_time" TEXT, 
	"general_info.difficulty" TEXT, 
	"general_info.serves" TEXT, 
	"general_info.description" TEXT, 
	"general_info.diet_type" TEXT, 
	"nutrition_info.kcal" TEXT, 
	"nutrition_info.fat" TEXT, 
	"nutrition_info.saturates" TEXT, 
	"nutrition_info.carbs" TEXT, 
	"nutrition_info.sugars" TEXT, 
	"nutrition_info.fibre" TEXT, 
	"nutrition_info.protein" TEXT, 
	"nutrition_info.salt" TEXT, 
	ingredients TEXT, 
	instructions TEXT
)

/*
3 rows from recipes table:
general_info.link	general_info.title	general_info.prep_time	general_info.cook_time	general_info.difficulty	general_info.serves	general_info.description	ge

In [14]:
response

{'input': 'givme a recipe with chicken in and spaghetti in it',
 'output': 'Here is a recipe that includes both chicken and spaghetti:\n\n- **Chicken Spaghetti**:\n  - **Prep time**: 20 mins\n  - **Cook time**: 40 mins\n  - **Difficulty**: Moderate\n  - **Serves**: 6\n  - **Description**: This delicious chicken spaghetti recipe is a family favorite.\n  - **Diet Type**: Not specified\n  - **Nutrition (per serving)**:\n    - **Calories**: 450 kcal\n    - **Fat**: 15g\n    - **Saturates**: 7g\n    - **Carbs**: 50g\n    - **Sugars**: 8g\n    - **Fibre**: 4g\n    - **Protein**: 30g\n    - **Salt**: 1.2g\n  - **Ingredients**:\n    - 1 lb chicken breast, diced\n    - 8 oz spaghetti\n    - 1 can cream of mushroom soup\n    - 1/2 cup grated Parmesan cheese\n    - 1/4 cup chopped parsley\n    - Salt and pepper to taste\n  - **Instructions**:\n    1. Cook spaghetti according to package instructions.\n    2. In a separate pan, cook diced chicken until browned.\n    3. Add cream of mushroom soup, P

In [11]:
response["output"]

"Here are 3 recipes that include chicken:\n\n1. **Creamy Tuscan Chicken**:\n   - **Prep time**: 10 mins\n   - **Cook time**: 30 mins\n   - **Difficulty**: Easy\n   - **Serves**: 4\n   - **Description**: Enjoy our quick midweek Tuscan chicken dinner, which makes the most of the color and flavor of spinach. Serve with spring greens or orzo.\n   - **Diet Type**: Not specified\n   - **Nutrition (per serving)**:\n     - **Calories**: 672 kcal\n     - **Fat**: 41g\n     - **Saturates**: 20g\n     - **Carbs**: 19g\n     - **Sugars**: 10g\n     - **Fiber**: 5g\n     - **Protein**: 49g\n     - **Salt**: 1.1g\n   - **Ingredients**:\n     - 2 tbsp plain flour\n     - 1 tbsp smoked paprika\n     - 2 tsp oregano\n     - 4 skin-on chicken breasts\n     - 80g sundried tomatoes, chopped, plus 1 tbsp oil from the jar (or use olive oil)\n     - 4 garlic cloves, finely grated or crushed\n     - 125ml white wine\n     - 250ml chicken stock\n     - 200g spinach\n     - 150ml double cream\n     - 75g grated